In [1]:
from pyspark import SparkContext, SparkConf
sc = SparkContext(appName='project')
sc

<SparkContext master=local[*] appName=project>

In [2]:
rdd = sc.sequenceFile('datasets/part-00000')

In [109]:
def word_counter(item):
    words_brute = item[1].strip().split() #melhorar
    words = [i.lower() for i in words_brute if (not any(j.isdigit() for j in i) and len(i) > 3)]
    if("hyundai" in words and "honda" in words):
        return [(i, 1) for i in words]
    return []

def document_counter(item):
    words_brute = item[1].strip().split() #melhorar
    words = [i.lower() for i in words_brute if (not any(j.isdigit() for j in i) and len(i) > 3)]
    if("hyundai" in words and "honda" in words):
        return [(i, 1) for i in set(words)]
    
    return []

def count_words(result ,item):
    return result+item
    
docs = rdd.flatMap(document_counter).reduceByKey(count_words)
words = rdd.flatMap(word_counter).reduceByKey(count_words)

In [110]:
print(docs.filter(lambda a: a[0] == "hyundai").collect())
print(docs.filter(lambda a: a[0] == "honda").collect())
print(words.filter(lambda a: a[0] == "hyundai").collect())
print(words.filter(lambda a: a[0] == "honda").collect())

[('hyundai', 242)]
[('honda', 242)]
[('hyundai', 847)]
[('honda', 1019)]


In [112]:
import math
import pandas as pd
N = docs.count()
    
def idf(item):
    return item[0], math.log(N/item[1],10)
    
def freq(item):
    return item[0], math.log(1 + item[1], 10)

def relevancia(item):
    return item[0], item[1][0] * item[1][1]

rdd_idf = docs.map(idf)

rdd_freq = words.map(freq)
rdd_relevancia = rdd_idf.join(rdd_freq).map(relevancia)

top_100 = rdd_relevancia.takeOrdered(100, key=(lambda x: -x[1]))
result = pd.DataFrame(top_100, columns=(['palavra', 'relevância']))
result.head(50)



,palavra,relevância
0,mecanico,13.038853
1,serie,12.349841
2,requerente:,11.574642
3,advogado(a):,11.407355
4,trator,11.265702
5,carregadeira,10.947045
6,requerido:,10.791081
7,automatico,10.469233
8,alcool,10.417174
9,disse:,10.310341
